# Primeiro trabalho de Algoritmos 2 - Geometria Computacional

Christian Rodrigues de Oliveira - 2019041817

Nesse trabalho serão abordados os aspectos práticos de diversos algoritmos de geometria computacional, especificamente serão explorados aspectos de implementação dos algoritmos para triangulação de polígonos aplicados ao problema da galeria de arte, que consiste em definir o menor número de câmeras de seguranças a serem colocadas em uma galeria a fim de cobrir todo o ambiente.

Dado um arquivo contendo vértices sequencialmente que formam um polígono simples, iremos realizar os seguintes passos:

1 - Realizar a triangulação do polígono;

2 - Realizar a coloração dos vértices;

A partir da coloração dos vértices nós sabemos o número mínimo de câmeras necessárias para cobrir toda a galeria, cada cor representará a posição de uma câmera e duas cores diferentes significam que você precisa de duas câmeras diferentes para cobrir esses vértices adjacentes.

In [200]:
pip install dash

In [201]:
pip install pandas

In [202]:
# Bibliotecas a serem utilizadas

import matplotlib.pyplot as plt
import sys
import plotly.graph_objects as go
import plotly.express as px

## Entrada de dados



iremos carregar um arquivo onde são passados vários vértices e existem arestas entre eles, sequencialmente definidos, formando um polígono.


In [203]:
# Lê os vértices de um polígono no formato float X Y
def vertices_from_file(arquivo):
    vertices = []
    with open(arquivo, 'r') as file:
        data = file.readline().strip().split()
        num_vertices = int(data[0])

        vertices = []
        for i in range(1, int(num_vertices)*2 , 2):
            if i != 0:
                x = float(data[i])
                y = float(data[i+1])

                vertices.append((x, y))
    return vertices


def plot_polygon(vertices):
  # Adiciona os vertices e garante que o último está conectado ao primeiro, para formar um polígono fechado.
    x = [float(vertex[0]) for vertex in vertices] + [float(vertices[0][0])]
    y = [float(vertex[1]) for vertex in vertices] + [float(vertices[0][1])]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', fill='toself', line=dict(color='cornflowerblue', width=2), marker=dict(color='cornflowerblue', size=6)))
    fig.update_layout(title='Polígono Original', xaxis_title='X', yaxis_title='Y')
    fig.show()

In [204]:
arquivo = 'sample_data/simple-20-1-new.pol'
vertices = vertices_from_file(arquivo)

plot_polygon(vertices)

## Triangulação do polígono

Primeiro iremos criar algumas funções auxiliares que serão úteis para a triangulação do polígono.

A triangulação do polígono é feita utilizando o método conhecido como corte de orelhas, esse método funciona com base na identificação e remoção de "orelhas" do polígono até que ele seja completamente decomposto em triângulos.

Uma orelha é um triângulo formado por três vértices consecutivos (P[i−1],P[i],P[i+1]) do polígono, onde nenhum outro vértice do polígono está dentro deste triângulo, e o ângulo interno do vértice P[i] é menor que 180 graus.

In [205]:
max_int = sys.maxsize
min_int = -sys.maxsize - 1

def intersect_point(Pi,Pj,Pk):
    '''
    Verifica a interseção entre 3 pontos
    '''

    if Pj[0] == Pi[0]:  # Caso onde a linha é vertical
        return (Pi[0], Pk[1])
    m = (Pj[1] - Pi[1]) / (Pj[0] - Pi[0])
    b = Pi[1] - m * Pi[0]
    y = m * Pk[0] + b
    return (Pk[0], y)


def direction(Pi, Pj, Pk):
    '''
    Determina o sentido de dois segmentos de reta consecutivos, PiPj e PjPk.
    Se negativo o sentido é anti-horario e se positivo é horario
    '''

    u = (Pk[0] - Pi[0], Pk[1] - Pi[1])
    v = (Pj[0] - Pi[0], Pj[1] - Pi[1])

    det = (u[0]*v[1]) - (u[1]*v[0])
    return det

def on_segment(Pi, Pj, Pk):
    '''
    Verifica se os pontos formam segmentos
    '''
    min_x = min(Pi[0],Pj[0])
    min_y = min(Pi[1],Pj[1])
    max_x = max(Pi[0],Pj[0])
    max_y = max(Pi[1],Pj[1])

    if min_x <= Pk[0] and Pk[0] <= max_x and min_y <= Pk[1] and Pk[1] <= max_y:
        return True
    else:
        return False

def segments_intersect(P1, P2, P3, P4):
    '''
    Verifica se há interseção de segmentos entre os vértices formados
    '''

    d1 = direction(P3,P4,P1)
    d2 = direction(P3,P4,P2)
    d3 = direction(P1,P2,P3)
    d4 = direction(P1,P2,P4)

    if ((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3 > 0 and d4 < 0) or (d3 < 0 and d4 > 0)):
        return True
    elif d1 == 0 and on_segment(P3,P4,P1):
        return True
    elif d2 == 0 and on_segment(P3,P4,P2):
        return True
    elif d3 == 0 and on_segment(P1,P2,P3):
        return True
    elif d4 == 0 and on_segment(P1,P2,P4):
        return True
    return False

def point_inside_polygon(P, q):
    '''
    Verifica se o ponto está dentro do polígono
    '''

    if q in P:
        return False

    count = 0
    polygon_size = len(P)
    up = True
    for i in P:
        if q[0] == i[0]:
            if q[1] < i[1]:
                up = True
            else:
                up = False
    for i in range(polygon_size):
        if not(up):
            up_line = (q[0], max_int)
            down_line = q
        else:
            up_line = q
            down_line = (q[0], min_int)

        if segments_intersect(down_line, up_line, P[i], P[(i + 1) % polygon_size]):
            y = intersect_point(P[i], P[(i + 1) % polygon_size], q)[1]
            if y < q[1]:
                count += 1
    return count % 2 == 1

In [206]:
def is_ear(P, V0, V1, V2):
    '''
    Verifica se os vértices formam orelhas
    '''
    point_inside = None
    for i in P:
        if point_inside_polygon([V0,V1,V2,V0],i):
            point_inside = True
            break
        else:
            point_inside = False
    if direction(V0,V1,V2) < 0:
        left = True
    else:
        left = False
    if point_inside == False and left == True:
        return True
    else:
        return False

def clipping(P):
    '''
    Realiza a triangulação e retorna as orelhas
    '''
    ears = []
    while len(P) > 3:
        for i in range(len(P)):
            if is_ear(P, P[i-1], P[i], P[(i+1)%len(P)]):
                ears.append((P[i-1],P[i],P[(i+1)%len(P)]))
                P.remove(P[i])
                break
    return (ears)


def plot_graph(triangulos):
    fig = go.Figure()

    # Plotar as arestas
    edges = graph.get_edges()
    for edge in edges:
        x_coords = [float(edge[0][0]), float(edge[1][0])]
        y_coords = [float(edge[0][1]), float(edge[1][1])]
        fig.add_trace(go.Scatter(x=x_coords, y=y_coords, mode='lines', line=dict(color='black')))

    # Plotar os vértices
    vertices = list(graph.graph.keys())
    x = [float(vertex[0]) for vertex in vertices]
    y = [float(vertex[1]) for vertex in vertices]
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(color='cornflowerblue', size=10)))

    fig.show()

Vamos definir o problema como um grafo, pois facilitará no restante da resolução:


In [207]:
class Graph:
    def __init__(self):
        self.graph = {}

    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = []

    def add_edge(self, v1, v2):
        if v1 in self.graph and v2 in self.graph:
            self.graph[v1].append(v2)
            self.graph[v2].append(v1)

    def get_edges(self):
        edges = []
        for vertex in self.graph:
            for neighbor in self.graph[vertex]:
                if (neighbor, vertex) not in edges:
                    edges.append((vertex, neighbor))
        return edges

    def get_vertex(self):
      list_vertex = []
      for vertex in self.graph:
        list_vertex.append(vertex)
      return list_vertex

    def __str__(self):
        return str(self.graph)

Agora iremos criar um método de plotar iterativamente as diagonais obtidas pela triangulação sobre o polígono, adicionando as diagonais a cada iteração.

In [208]:
# Função para plotar o grafo iterativamente
def plot_graph_iteratively(graph, triangulos):

    # Função para adicionar arestas ao gráfico
    def add_edge_trace(edge, fig, color='black'):
        x_coords = [float(edge[0][0]), float(edge[1][0])]
        y_coords = [float(edge[0][1]), float(edge[1][1])]
        fig.add_trace(go.Scatter(x=x_coords, y=y_coords, mode='lines', line=dict(color=color)))

    # Inicializar as estruturas para os frames
    frames = []
    vertices = graph.get_vertex()

    # Frame inicial com apenas as arestas e vértices do polígono original
    fig = go.Figure()

    for edge in graph.get_edges():
        add_edge_trace(edge, fig)

    # Adicionar os vértices iniciais
    x = [float(vertex[0]) for vertex in vertices]
    y = [float(vertex[1]) for vertex in vertices]

    fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(color='cornflowerblue', size=10)))

    # Adicionar o frame inicial
    frames.append(go.Frame(data=fig.data, name='frame0'))

    # Iterativamente adicionar diagonais ao polígono
    for idx, tri in enumerate(triangulos):
        # Adiciona a diagonal (orelha) ao grafo
        graph.add_edge(tri[0], tri[2])

        # Criar novo frame
        frame_data = []

        # Mantém o vértices dos grafos
        frame_data.append(go.Scatter(x=x, y=y, mode='markers', marker=dict(color='cornflowerblue', size=10)))

        # Mantém as arestas do grafo inicial
        add_edge_trace(edge, fig)

        # Adiciona aresta nova a cada frame
        for edge in graph.get_edges():

            frame_data.append(go.Scatter(
                x=[float(edge[0][0]), float(edge[1][0])],
                y=[float(edge[0][1]), float(edge[1][1])],
                mode='lines', line=dict(color='black')
            ))

        # Adicionar novo frame à lista de frames
        frames.append(go.Frame(data=frame_data, name=f'frame{idx+1}'))

    # Configurar o layout e os controles de animação
    fig.frames = frames
    fig.update_layout(
        title='Adição das diagonais de triangulação no polígono',
        xaxis_title='X',
        yaxis_title='Y',
        updatemenus=[{
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }],
        sliders=[{
            'steps': [
                {'args': [[f'frame{idx}'], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                'label': f'{idx}', 'method': 'animate'}
                for idx in range(len(frames))
            ],
            'transition': {'duration': 0},
            'x': 0.1,
            'len': 0.9
        }]
    )

    fig.show()

In [209]:
# Criando grafo com o polígono existente
graph = Graph()
for v in vertices:
    graph.add_vertex(v)

for i in range(len(vertices)):
    graph.add_edge(vertices[i], vertices[(i + 1) % len(vertices)])

# Realiza a triangulação e adiciona diagonais ao grafo
P = vertices[:]
triangulos = clipping(P)

plot_graph_iteratively(graph, triangulos)

## Coloração de vértices

Realizaremos o processo de atribuição de cores a cada vértice do grafo de modo que dois vértices adjacentes não compartilhem a mesma cor.

In [210]:
def coloration_set(graph):
    '''
    Dado um grafo, aplica o algoritmo de coloração e retorna as cores de cada vértice
    '''

    # Inicializa as cores dos vértices como não atribuídas
    colors = {v: -1 for v in graph.get_vertex()}

    # Ordena os vértices pelo número de arestas
    vertices = sorted(graph.get_vertex(), key=lambda v: len(graph.graph[v]), reverse=True)

    for vertex in vertices:
        # Obtem as cores dos vizinhos
        colors_neighbor = set(colors[neighbor] for neighbor in graph.graph[vertex] if colors[neighbor] != -1)

        # Atribui a menor cor disponível
        cor = 0
        while cor in colors_neighbor:
            cor += 1
        colors[vertex] = cor

    return colors

def min_cameras(colors):
    '''
    Retorna no número de cores utilizadas
    '''
    print ("Número mínimo de câmeras para cobrir o polígono: ")
    return len(set(colors.values()))

def plot_colored_graph(graph, coloracao):
    '''
    Função para plotar o polígono colorido, dado um grafo e as cores de seus vértices
    '''
    fig = go.Figure()

    color_map = {
        -1: 'white',
        0: 'green',
        1: 'red',
        2: 'yellow',
        3: 'blue',
        4: 'purple'
    }

    # Plotar as arestas
    edges = graph.get_edges()
    for edge in edges:
        x_coords = [float(edge[0][0]), float(edge[1][0])]
        y_coords = [float(edge[0][1]), float(edge[1][1])]
        fig.add_trace(go.Scatter(x=x_coords, y=y_coords, mode='lines', line=dict(color='black'), showlegend=False))

    # Plotar os vértices com suas cores atribuídas
    vertices = graph.get_vertex()
    x = [float(vertex[0]) for vertex in vertices]
    y = [float(vertex[1]) for vertex in vertices]

    # Gerar a legenda de cores
    legend_added = set()  # Para rastrear quais cores já foram adicionadas à legenda

    # Plotar os vértices com cores e legenda das cores
    for i, vertex in enumerate(vertices):
        color = coloracao[vertex]
        color_value = color_map.get(color, 'white')  # Cor padrão para valores não mapeados

        showlegend = False
        if color not in legend_added:
            showlegend = True
            legend_added.add(color)

        fig.add_trace(go.Scatter(x=[x[i]], y=[y[i]], mode='markers', marker=dict(color=color_value, size=10), name=f'Cor {color}' if showlegend else None, showlegend=showlegend))
    fig.update_layout(title='Grafo do Polígono Colorido', xaxis_title='X', yaxis_title='Y')
    fig.show()

In [211]:
coloracao = coloration_set(graph)

plot_colored_graph(graph, coloracao)

In [212]:
min_cameras(coloracao)

Número mínimo de câmeras para cobrir o polígono: 


3

O número mínimo de câmeras necessárias para resolver o problema é igual ao número de cores diferentes usadas na coloração do grafo.

Como podemos ver, para esse polígono o número mínimo de câmeras necessárias para cobrir todo o polígono é 3.

## Analisando outro polígono


In [213]:
arquivo = 'sample_data/simple-30-1-new.pol'
vertices = vertices_from_file(arquivo)

plot_polygon(vertices)
# Criando grafo com o polígono existente
graph = Graph()
for v in vertices:
    graph.add_vertex(v)

for i in range(len(vertices)):
    graph.add_edge(vertices[i], vertices[(i + 1) % len(vertices)])

# Realiza a triangulação e adiciona diagonais ao grafo
P = vertices[:]
triangulos = clipping(P)

plot_graph_iteratively(graph, triangulos)
coloracao = coloration_set(graph)

plot_colored_graph(graph, coloracao)

min_cameras(coloracao)

Número mínimo de câmeras para cobrir o polígono: 


4